<h1 style="color:blue;">Group 4 Final Project</h1>


https://mesonet.agron.iastate.edu/request/download.phtml?network=NJ_ASOS

<font size="5">Members:</font> <br> Birendra Khimding <br> Muzhgan Rustaqi <br> Andrew Fennimore

In [63]:
import pandas as pd
import numpy as np


# Read Data from the MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/refs/heads/main/MJX.csv"

#All values with na are labeled as M
weather = pd.read_csv("MJX.csv" , na_values= "M")


In [ ]:
ordinal_mapping = {
    'CLR': 0,
    'FEW': 1,
    'SCT': 2,
    'BKN': 3,
    'OVC': 4,
    'VV': 5
}

weather['cloud_ordinal']= weather['cloud_coverage'].map(ordinal_mapping)
weather['cloud_ordinal'].value_counts()
weather = weather.dropna(subset=['cloud_ordinal'])
weather.isna().sum()

In [18]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Read Data from the MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/main/MJX.csv"

# All values with na are labeled as M
weather = pd.read_csv(url, na_values="M")

# List of variables to fill missing values with their mean
variables_to_fill = ["vsby", "tmpf", "relh", "drct", "sknt", "alti", "mslp", "gust", 
                     "peak_wind_gust", "peak_wind_drct"]

# Fill missing values with the mean for specified variables
for variable in variables_to_fill:
    weather[variable].fillna(weather[variable].mean(), inplace=True)

# Model 1: Predict visibility using various weather variables
model = smf.ols(formula="vsby ~ tmpf + relh + drct + sknt + alti + mslp + gust + \
                        skyl1 + skyl2 + skyl3 + C(skyc1) + \
                        peak_wind_gust + \
                        peak_wind_drct", 
                data=weather).fit()

# Print summary of the model
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                   vsby   R-squared:                       0.211
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     15.05
Date:                Sat, 22 Feb 2025   Prob (F-statistic):           1.43e-32
Time:                        20:48:23   Log-Likelihood:                -1861.9
No. Observations:                 805   AIC:                             3754.
Df Residuals:                     790   BIC:                             3824.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           8.7004     19.056     

C:\Users\fenn3\AppData\Local\Temp\ipykernel_6364\2899200562.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather[variable].fillna(weather[variable].mean(), inplace=True)


In [65]:

# Define a constant K greater than the maximum value of vsby
K = weather['vsby'].max() + 1

# Apply the reverse log transformation and create the GLM model
modelglm_neg = smf.glm(formula=f"np.log({K} - vsby) ~ tmpf + dwpf + relh + drct + sknt + alti + mslp + gust + \
                        (skyl1 * skyl2 * skyl3) + \
                        peak_wind_gust + \
                        feel", 
                data=weather).fit()

print(modelglm_neg.summary())


                  Generalized Linear Model Regression Results                  
Dep. Variable:     np.log(11.0 - vsby)   No. Observations:                11423
Model:                             GLM   Df Residuals:                    11407
Model Family:                 Gaussian   Df Model:                           15
Link Function:                Identity   Scale:                         0.35815
Method:                           IRLS   Log-Likelihood:                -10336.
Date:                 Sat, 22 Feb 2025   Deviance:                       4085.4
Time:                         18:53:59   Pearson chi2:                 4.09e+03
No. Iterations:                      3   Pseudo R-squ. (CS):             0.4124
Covariance Type:             nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             3.0874

In [90]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Read Data from the MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/main/MJX.csv"

# All values with na are labeled as M
weather = pd.read_csv(url, na_values="M")

# List of significant variables to fill missing values with their mean
significant_variables = ["vsby", "tmpf", "dwpf", "relh", "drct", "skyl1", "skyl3", "feel", "mslp"]

# Fill missing values with the mean for specified significant variables
for variable in significant_variables:
    weather[variable].fillna(weather[variable].mean(), inplace=True)

# Create a new model using only significant variables
model_sig = smf.ols(formula="vsby ~ tmpf + relh + drct + c(skyl1) + c(skyl3) + feel + mslp", 
                    data=weather).fit()

# Print summary of the new model
print(model_sig.summary())


                            OLS Regression Results                            
Dep. Variable:                   vsby   R-squared:                       0.303
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     621.6
Date:                Sat, 22 Feb 2025   Prob (F-statistic):               0.00
Time:                        19:07:33   Log-Likelihood:                -25026.
No. Observations:               11423   AIC:                         5.007e+04
Df Residuals:                   11414   BIC:                         5.014e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1179      3.401      0.329      0.7

C:\Users\fenn3\AppData\Local\Temp\ipykernel_2988\864213903.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather[variable].fillna(weather[variable].mean(), inplace=True)


In [120]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Read Data from the MJX
url = "https://raw.githubusercontent.com/Fenn3963/Weather-Impact-on-Air-Traffic-Management/main/MJX.csv"

# All values with na are labeled as M
weather = pd.read_csv(url, na_values="M")

# List of significant variables to fill missing values with their mean
significant_variables = ["vsby", "tmpf", "dwpf", "relh", "drct", "skyl1", "skyl3", "feel", "mslp"]

# Fill missing values with the mean for specified significant variables
for variable in significant_variables:
    weather[variable].fillna(weather[variable].mean(), inplace=True)

# Fit the GLM model with Gamma family and log link
model_gamma = smf.glm(formula="vsby ~ tmpf + relh + drct + skyl1 + skyl3 + feel + mslp", 
                      data=weather,
                      family=sm.families.Gamma(link=sm.families.links.log())).fit()

# Print summary of the GLM model
print(model_gamma.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   vsby   No. Observations:                11423
Model:                            GLM   Df Residuals:                    11415
Model Family:                   Gamma   Df Model:                            7
Link Function:                    log   Scale:                        0.091943
Method:                          IRLS   Log-Likelihood:                -33011.
Date:                Sat, 22 Feb 2025   Deviance:                       2271.1
Time:                        19:24:05   Pearson chi2:                 1.05e+03
No. Iterations:                    18   Pseudo R-squ. (CS):             0.2291
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.0682      0.460     -4.492      0.0

C:\Users\fenn3\AppData\Local\Temp\ipykernel_2988\2739636273.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather[variable].fillna(weather[variable].mean(), inplace=True)
C:\Users\fenn3\anaconda3\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
